<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2022_ex05a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2022-ex05a

課題の期限や提出の方法などについては，Visionチーム内に書いてます．


## PyTorch でロジスティック回帰

この課題とそれに続く課題では，[PyTorch](https://pytorch.org/) という深層学習フレームワークを用いて，ニューラルネットの学習の実験をやってみます．まずは，ニューラルネットの前に，ロジスティック回帰から



In [ ]:
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn

### Dataset 

PyTorch のプログラムでは，データを Dataset クラス（torch.utils.data.Dataset）のオブジェクトとして扱うと便利．
ここでは，MNIST データのクラスとしてすでに用意されている [torchvision.datasets.MNIST](https://pytorch.org/vision/stable/datasets.html#mnist) クラスを使ってみる．
自前のデータセットを使う場合は，Dataset クラスを拡張して自分のクラスを作ればよい．

MNIST クラスのインスタンスを生成．学習データとテストデータ．
以下のセルを初めて実行したときは，カレントディレクトリの下の `data/MNIST` 以下に MNIST のデータがダウンロードされる．2回目以降はデータが存在していれば再ダウンロードはしない．

`transform` オプションに指定しているものの意味については，今の時点ではごく簡単に．

PyTorch では，データを **Tensor** という形式で扱う．NumPy の array と同じような配列と思ってよい．数学の概念である「テンソル」に似てなくもない．Dataset クラスでは，`transform` オプションにいろいろ指定することで，例えば画像だったら拡大縮小したり一部を切り取ったりと様々な前処理を施すことができるようになっている．
ここでは，最小限の前処理として，ファイルから読み込んだ画素値を Tensor に変換するよう指示している．

In [ ]:
# 学習データのインスタンス
dsL = MNIST(root='data', train=True, download=True, transform=ToTensor())

# テストデータのインスタンス
dsT = MNIST(root='data', train=False, download=True, transform=ToTensor())

`dsL` の最初の `N` 個のデータを取り出して眺めてみる．

In [ ]:
N = 10

fig = plt.figure(figsize=(10, 2))
for n in range(N):
    x, y = dsL[n]  # n 番目の学習データを取り出す
    fig.add_subplot(1, N, n+1)
    plt.title(f'class{y}')
    plt.axis('off')
    plt.imshow(x.squeeze(), cmap='gray')

plt.show()

最初の要素を取り出してみる．`x` は 32bit 浮動小数点数を要素にもつ 1x28x28 のテンソル．画素値が 0 から 1 の値で入っている．
`y` はクラスラベルを表す整数値．

In [ ]:
x, y = dsL[0]
print(x.shape, x.dtype)
print(y)

In [ ]:
x

### Dataloader

機械学習では，大量の学習データを扱うことになるので，すべてを一度にメモリに読み込んでしまうのではなく，少しずつ読み込みながら処理したい場面が多い．また，学習データをランダムな順番で扱ったりもしたい，そういうことが簡単にできるように，PyTorh には [torch.utils.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) というクラスが用意されている．

DataLoader のインスタンスを生成する際は，Dataset のインスタンスをコンストラクタの引数に指定する．MNIST でも自前のデータでも，Dataset クラスのサブクラスとして定義しておけば，DataLoader で扱える．

In [ ]:
# 学習データ
dlL = DataLoader(dsL, batch_size=10, shuffle=True)

# テストデータ
dlT = DataLoader(dsT, batch_size=10, shuffle=False)

以下のセルを何度か実行してみよう．実行するたびに，`dsL` から `batch_size` 個のデータが取り出されて `X` と `Y` に代入される．

In [ ]:
X, Y = next(iter(dlL)) # dlL から一つのデータバッチを取り出す
print(X.shape)
print(Y.shape)
print(Y)

次は，`dlT` の方で同じことをしてみよう．

In [ ]:
X, Y = next(iter(dlT)) # dlT から一つのデータバッチを取り出す
print(X.shape)
print(Y.shape)
print(Y)

`dlT` の方は，`shuffle=False` で生成した DataLoader なので，データを毎回同じ順序で取り出すことになる．上記では毎回イテレータを初期化しているので，実行するたびに `dsT` の先頭から 10 個が取り出される → 毎回同じデータが出てくる，ちうことになる．

### ロジスティック回帰のネットワークモデルの定義とインスタンスの生成

PyTorch で深層学習のプログラムを作る際は，[torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) のサブクラスとしてニューラルネットワークの構造を定義する．
ここでは，ロジスティック回帰モデルを定義してみる．ロジスティック回帰は，一層しかない（中間層のない）最も単純な構造のニューラルネットとみなすことができる．

- [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
- [torch.nn.LogSoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html)

In [ ]:
class Logistic(nn.Module):

    # いわゆるコンストラクタ
    #    D: 入力の大きさ（データの次元数）  K: 出力の大きさ（クラス数）
    def __init__(self, D, K):

        # スーパークラスのコンストラクタ呼び出し
        super(Logistic, self).__init__()

        # インスタンス変数
        self.D = D
        self.K = K

        # 全結合層の定義
        self.fc = nn.Linear(D, K)

        # softmax（のlog）の定義
        self.logsoftmax = nn.LogSoftmax(dim=1)

    # 出力を計算するインスタンスメソッドの定義．nn.Module の forward メソッドをオーバーライド
    def forward(self, X):

        X = self.fc(X)
        X = self.logsoftmax(X)
        return X

現在の環境で GPU を使えるか（CUDAの使えるデバイスがあるかどうか）を調べる．PyTorch で書いたプログラムは，CUDA が使える環境ならそのまま GPU で動かすことができる．

Colab はデフォルトでは CPU で動くが，上部のメニューで「ランタイム」> 「ランタイムのタイプを変更」から，「ハードウェアアクセラレータ」に「GPU」を指定すれば，GPUで動かすこともできる．ランタイムを再起動することになるので，一からセルを実行し直す必要あり．

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')

上記で定義した `Logistic` クラスのインスタンスを生成．
入力がD次元，出力がK次元でバイアスありの Linear 層ひとつだけのネットワーク．
ネットワークのパラメータは適当な乱数で初期化される．

In [ ]:
D, K = 100, 3
model = Logistic(D, K).to(device)
print(model)

ためしに 5xD のデータを乱数で作り，モデルに入力してみる．出力が 5xK になっていることがわかる．

In [ ]:
X = torch.rand(5, D).to(device)
Y = model(X)
print(X.shape, Y.shape)

### 学習（パラメータ最適化）のための準備

PyTorch では，ネットワークモデルのパラメータについて，損失関数のパラメータでの微分を自動的に計算してくれる．勾配を手計算してそれをコーディングする，という手間が不要．

以下のセルを実行すると，次の二つのことが行われる．
- 損失関数の定義．[torch.nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
- `model` のパラメータに対して学習定数 0.01 で Stochastic Gradient Descent する最適化器を用意．
[torch.optim.SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)

In [ ]:
# 損失関数を用意．Negative Log-Likelihood = 負の対数尤度 = 交差エントロピー
loss_func = nn.NLLLoss(reduction='sum')

# 最適化器を用意．SGD
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

SGD だけでなくもっと凝った最適化アルゴリズムもいろいろ使用可能．https://pytorch.org/docs/stable/optim.html

### 学習

ここからは，上記で説明していたコードを改めて書いて，MNISTの識別を学習させてみましょう．

学習データの用意

In [ ]:
D = 28*28 # データの次元数
K = 10    # クラス数

# データの用意
dsL = MNIST(root='data', train=True, download=True, transform=ToTensor())
dlL = DataLoader(dsL, batch_size=100, shuffle=True)

# 学習モデルの生成
model = Logistic(D, K).to(device)
print(model)

# 損失関数を用意．Negative Log-Likelihood = 負の対数尤度 = 交差エントロピー
loss_func = nn.NLLLoss(reduction='sum')

# 最適化器を用意．SGD
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

次のセルが学習のループ．深層学習の界隈では，個々の学習データを一回ずつ学習するサイクルを「**epoch**（エポック）」という．以下では学習を 10 エポック繰り返している．

この例では，学習データ数が 60000 で batchsize が 100 なので，`for i` のループは 60000 / 100 = 600 回繰り返している．つまり 1 エポックの間にパラメータを 600 回更新している．

In [ ]:
nepoch = 10

for t in range(nepoch):

    loss_sum = 0.0
    n = 0

    for i, (X, lab) in enumerate(dlL):

        # device で指定されたデバイスへ転送
        X = X.to(device)
        lab = lab.to(device)

        # X を (batchsize, 1, 28, 28) から (batchsize, 784) へ reshape
        X = X.reshape((-1, D)).to(device)

        Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = loss_func(Y, lab) # 正解ラベル Zt に対する loss を計算

        optimizer.zero_grad()  # 勾配をリセット
        loss.backward()        # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()       # パラメータを更新

        n += len(X)
        loss_sum += loss.item()  # 損失関数の値


    print(f'{t} {loss_sum/n}')


### 学習後の loss と識別率を算出

学習データに対する loss と識別率．

In [ ]:
# ネットワークを eval mode にする
model.eval()

loss_sum = 0.0
ncorrect = 0
n = 0

for i, (X, lab) in enumerate(dlL):

    # device で指定されたデバイスへ転送
    X = X.to(device)
    lab = lab.to(device)

    # X を (batchsize, 1, 28, 28) から (batchsize, 784) へ reshape
    X = X.reshape((-1, D))

    Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
    loss = loss_func(Y, lab) # 正解ラベル Zt に対する loss を計算

    n += len(X)
    loss_sum += loss.item()  # 損失関数の値
    ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

print(f'# L: loss = {loss_sum/n:.4f}  ncorrect/n = {ncorrect}/{n} = {ncorrect/n:.4f}')


テストデータに対する loss と識別率

In [ ]:
# Dataset & DataLoader
dsT = MNIST(root='data', train=False, download=True, transform=ToTensor())
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

# ネットワークを eval mode にする
model.eval()

loss_sum = 0.0
ncorrect = 0
n = 0

for i, (X, lab) in enumerate(dlT):

    # device で指定されたデバイスへ転送
    X = X.to(device)
    lab = lab.to(device)

    # X を (batchsize, 1, 28, 28) から (batchsize, 784) へ reshape
    X = X.reshape((-1, D))

    Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
    loss = loss_func(Y, lab) # 正解ラベル Zt に対する loss を計算

    n += len(X)
    loss_sum += loss.item()  # 損失関数の値
    ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

print(f'# L: loss = {loss_sum/n:.4f}  ncorrect/n = {ncorrect}/{n} = {ncorrect/n:.4f}')